In [1]:
import torch
import torchaudio

torch.random.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"

# print(torch.__version__)
# print(torchaudio.__version__)
# print(device)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/user/codec/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/user/codec/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/user/codec/.venv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/Users/u

In [2]:
# symbols = "_-!'(),.:;? abcdefghijklmnopqrstuvwxyz"
# look_up = {s: i for i, s in enumerate(symbols)}
# symbols = set(symbols)


# def text_to_sequence(text):
#     text = text.lower()
#     return [look_up[s] for s in text if s in symbols]


# text = "Hello world! Text to speech!"
# print(text_to_sequence(text))

In [3]:
# import torch
# import torch.nn as nn

# class Encoder(nn.Module):
#     def __init__(self, input_dim, hidden_dim):
#         super(Encoder, self).__init__()
#         self.rnn = nn.LSTM(input_dim, hidden_dim, batch_first=True)
    
#     def forward(self, x):
#         outputs, (hidden, cell) = self.rnn(x)
#         return outputs, hidden, cell

# class Attention(nn.Module):
#     def __init__(self, hidden_dim):
#         super(Attention, self).__init__()
#         self.attention = nn.Linear(hidden_dim, hidden_dim)
    
#     def forward(self, encoder_outputs, hidden):
#         scores = torch.bmm(encoder_outputs, hidden.unsqueeze(2)).squeeze(2)
#         attention_weights = torch.softmax(scores, dim=1)
#         context = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs).squeeze(1)
#         return context

# class Decoder(nn.Module):
#     def __init__(self, hidden_dim, output_dim):
#         super(Decoder, self).__init__()
#         self.rnn = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, output_dim)
    
#     def forward(self, x, context):
#         rnn_output, _ = self.rnn(x)
#         output = self.fc(rnn_output + context.unsqueeze(1))
#         return output

# class TTSModel(nn.Module):
#     def __init__(self, input_dim, hidden_dim, output_dim):
#         super(TTSModel, self).__init__()
#         self.encoder = Encoder(input_dim, hidden_dim)
#         self.attention = Attention(hidden_dim)
#         self.decoder = Decoder(hidden_dim, output_dim)
    
#     def forward(self, text_features, mel_spectrogram_target):
#         encoder_outputs, hidden, cell = self.encoder(text_features)
#         context = self.attention(encoder_outputs, hidden[-1])
#         output = self.decoder(mel_spectrogram_target, context)
#         return output


In [4]:
# train_dataset = torchaudio.datasets.LJSPEECH(root='dataset', download=True)
# test_dataset = torchaudio.datasets.LJSPEECH(root='dataset', download=True)
train_dataset = torchaudio.datasets.CMUARCTIC(root='dataset', download=True)
test_dataset = torchaudio.datasets.CMUARCTIC(root='dataset', download=True)
waveform, sample_rate, transcript, utterance_id = train_dataset[0]

In [5]:
# waveform, sample_rate, utterance, speaker_id, chapter_id, utterance_id = train_dataset[0]
# waveform, sample_rate, transcript, utterance_id = train_dataset[0]
# print(f'Waveform shape: {waveform.shape}')
# print(f'Sample rate: {sample_rate}')
# print(f'Utterance: {utterance_id}')

In [6]:
# from torch.utils.data import DataLoader
# 
# train_loader = DataLoader(train_dataset, shuffle=True)
# for batch in train_loader:
#     print(batch)
#     waveform, sample_rate, transcript, utterance_id = batch

In [7]:
# import torchaudio.transforms as T
# # 1. Data Preprocessing


# def preprocess_text(transcript):
#     return torch.tensor(text_to_sequence(transcript), dtype=torch.long)

# def preprocess_audio(waveform, sample_rate):
#     mel_spectrogram_transform = T.MelSpectrogram(
#         sample_rate=sample_rate,
#         n_fft=1024,
#         hop_length=256,
#         n_mels=80
#     )
#     mel_spectrogram = mel_spectrogram_transform(waveform)
#     return mel_spectrogram

In [8]:
import torchaudio
import torchaudio.transforms as T
import torch
# from sklearn.preprocessing import LabelEncoder
import numpy as np

# Load the dataset
train_dataset = torchaudio.datasets.CMUARCTIC(root='dataset', download=True)
test_dataset = torchaudio.datasets.CMUARCTIC(root='dataset', download=True)

# Example preprocessing
def preprocess_data(dataset):
    audio_data = []
    text_data = []
    for waveform, sample_rate, transcript, _ in dataset:
        # Convert waveform to mel spectrogram
        mel_spectrogram = T.MelSpectrogram()(waveform)
        audio_data.append(mel_spectrogram)
        
        # Encode text data (simple character encoding for demo)
        text_data.append(transcript)
    
    return audio_data, text_data

train_audio_data, train_text_data = preprocess_data(train_dataset)
test_audio_data, test_text_data = preprocess_data(test_dataset)


/Users/user/codec/.venv/lib/python3.12/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [9]:
import torch.nn as nn

class SimpleTTSModel(nn.Module):
    def __init__(self):
        super(SimpleTTSModel, self).__init__()
        self.lstm = nn.LSTM(input_size=80, hidden_size=256, num_layers=2, batch_first=True)
        self.fc = nn.Linear(256, 80)  # Output should match mel spectrogram dimensions

    def forward(self, x):
        h, _ = self.lstm(x)
        out = self.fc(h)
        return out

model = SimpleTTSModel()


In [10]:
import torch
import torch.nn.functional as F

def pad_sequence(sequences, max_length):
    padded_sequences = []
    for seq in sequences:
        if seq.size(1) < max_length:
            # Pad the sequence with zeros
            padding_size = max_length - seq.size(1)
            padded_seq = F.pad(seq, (0, padding_size), mode='constant', value=0)
        else:
            # Truncate the sequence if it's too long
            padded_seq = seq[:, :max_length]
        padded_sequences.append(padded_seq)
    return torch.stack(padded_sequences)

# Determine max length
max_length = max([s.size(1) for s in train_audio_data])

# Modify your DataLoader to pad sequences
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, audio_data, text_data, max_length):
        self.audio_data = audio_data
        self.text_data = text_data
        self.max_length = max_length

    def __len__(self):
        return len(self.audio_data)

    def __getitem__(self, idx):
        mel_spectrogram = self.audio_data[idx]
        text = self.text_data[idx]
        mel_spectrogram = pad_sequence([mel_spectrogram], self.max_length)[0]  # Pad the spectrogram
        return mel_spectrogram, text

train_dataset = CustomDataset(train_audio_data, train_text_data, max_length)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
# Define a custom collate function for dynamic padding
# from torch.nn.utils.rnn import pad_sequence
import torch
import numpy as np


class CollateFn:
    def __init__(self):
        pass

    def __call__(self, batch):
        mel_spectrograms, texts = zip(*batch)

        # Ensure all mel spectrograms have the same number of features
        num_features = mel_spectrograms[0].size(2)  # Get number of features from the first mel spectrogram

        # Find the maximum length in the batch
        max_length = max(mel.size(1) for mel in mel_spectrograms)

        # Pad mel spectrograms
        padded_mel_spectrograms = []
        for mel in mel_spectrograms:
            current_length = mel.size(1)
            if current_length < max_length:
                # Create a tensor of zeros with the desired shape
                padded_mel = torch.zeros((mel.size(0), max_length, num_features))
                # Copy the original mel spectrogram into the padded tensor
                padded_mel[:, :current_length, :] = mel
            else:
                # Use original tensor if it’s already at max_length
                padded_mel = mel
            
            padded_mel_spectrograms.append(padded_mel)
        
        # Ensure all tensors in padded_mel_spectrograms are of the same shape before stacking
        if not all(padded_mel.size(1) == max_length for padded_mel in padded_mel_spectrograms):
            raise RuntimeError("Inconsistent tensor sizes after padding.")

        # Stack the list of padded tensors into a single tensor
        mel_spectrograms = torch.stack(padded_mel_spectrograms)
        
        return mel_spectrograms, texts



def train(model, data_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        for mel_spectrograms, texts in data_loader:
            optimizer.zero_grad()
            outputs = model(mel_spectrograms)
            
            # Ensure the outputs and targets are the same shape
            targets = mel_spectrograms
            if outputs.shape != targets.shape:
                # Truncate or pad the outputs to match the targets (for this example)
                outputs = outputs[:, :targets.size(1), :]
            
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')



# Create the DataLoader with the custom collate function
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=CollateFn())

# Example model, criterion, and optimizer setup
model = SimpleTTSModel()  # Define your model
criterion = nn.MSELoss()  # Define loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Define optimizer

# Train the model
train(model, train_loader, criterion, optimizer)


RuntimeError: stack expects each tensor to be equal size, but got [1, 128, 161] at entry 0 and [1, 128, 274] at entry 1

In [ ]:
# def evaluate_model(model, test_loader):
#     model.eval()
#     total_correct = 0
#     total_samples = 0
#     with torch.no_grad():
#         for mel_spectrograms, transcripts in test_loader:
#             outputs = model(mel_spectrograms)
#             _, predicted = torch.max(outputs, 1)
#             total_samples += transcripts.size(0)
#             total_correct += (predicted == transcripts).sum().item()
    
#     accuracy = total_correct / total_samples
#     print(f'Test Accuracy: {accuracy:.4f}')

# evaluate_model(model, test_loader)

In [ ]:
# import torchaudio.transforms as T

# def generate_speech(model, text):
#     model.eval()
#     with torch.no_grad():
#         # Simple example, assume `text` has been preprocessed to a suitable tensor
#         mel_spectrogram = model(text)
#         # Convert mel spectrogram back to waveform (placeholder for inverse operation)
#         waveform = T.InverseMelScale(n_stft=512)(mel_spectrogram)
#         return waveform

# # Generate speech for a new text example
# text_example = torch.randn(1, 50, 80)  # Replace with actual text preprocessing
# generated_waveform = generate_speech(model, text_example)
# torchaudio.save('generated_speech.wav', generated_waveform, sample_rate=22050)
